# ECM2423 - Artificial Intelligence and Applications
## Coursework exercise (This notebook contains Q1-Q1.3)
### Student No: 700074704
***

### Question 1 | Implement a heuristic search algorithm: A⋆ and the 8-puzzle game 

#### Question 1.1: Describe how you would frame the 8-puzzle problem as a search problem.</h4>
***

**The main objective of the 8-puzzle problem is to achieve a final configuration of the titles from the given initial configuration.**

It can be defined as a search problem due to its following properties:

**A set of states**: The puzzle has a number of different states which depend on the integer locations of the titles.

**Operators**: The puzzle has 4 operators: left, right, up or down. As one title is missing there is the ability to move one of the adjacent titles into that position.

**A goal state**: Reaching the final given configuration is the end goal state. This goal state depends on the pre-defined configuration. For figure 1, the goal state should contain the missing title in the top right corner, with the remaining titles in ascending order. 

**Path cost**: A search problem also has a path cost, for the 8-puzzle it is 1 per move made. 

---

<h4> Question 1.2: Solve the 8-puzzle problem using A⋆ </h4>

<h4>1.2.1 In this question, you should first briefly outline the A⋆
algorithm</h4>

---

The A* Algorithm is a searching algorithm that works to achieve the lowest cost path between two (initial and final) nodes. It uses a heuristic function wherein it decides which path to follow next based on choosing the node which has the lower weight. In this way it is able to give an estimate of the minimum cost between the initial and the target node.  

$f(n)=g(n)+h(n)$

The algorithm consists of 3 parameters:

* $f(n)$ : refers to the weight of the nodes, i.e the total estimated cost of path via node $n$. It is the sum of two numbers ('g' and 'h').


* $g(n)$ : refers to the cost so far to reach the current node $n$ (from the initial node/ root state). It ensures that all the higher nodes are traversed. For e.g. the 8-puzzle begins with two possible moves which can be made. This 'g’ number makes sure to traverse both of these moves rather than choosing one and attemping to complete the rest of the puzzle from it.


* $h(n)$ : the estimated cost from $n$ to the target node. It refers to the heuristic part of the cost function, i.e. it is a guess of how close the current node is to the final target node. There are various ways in which to calculate $h$, therefore a good heuristic can affect the performance of an A* search considerably. In other words, it will result in an optimal search if the heuristic is admissible. 



<h4> 1.2.2. Describe two admissible heuristic functions for the 8-puzzle problem and explain why they are admissible. Make sure you also explain why you chose these two heuristic functions in particular amongst all the possible ones. </h4>

The two admissible heuristic functions I have chosen are the Manhattan and Euclidean. They are admissible because the cost which they express is always going to be smaller than the real cost to the goal. 

Admissible heuristics do not overestimate the number of moves to solve a problem. In this puzzle, you are only able to move the block one at a time, and in 4 directions such that in each move it is closer to the final configuration by one position. Once the minimum weighting has been achived (0), the puzzle is solved.

The Manhattan distance is the sum of the distances of each tile to their goal state. Therefore the heuristic value will get smaller if the tiles are closer to their goal position. In an optimal scenario, each block has a clear path to its goal state, and this is known as an M.D. (Manhattan Distance) of 1. The rest of the states would require a greater number moves than the M.D. to reach the goal state, so would be sub-optimal. Therefore the Manhattan Distance does not overestimate and is admissible.

For the Manhattan distance, we sum the horizontal (rows) and vertical (columns) to the target, whereas for Eucledian we take the direct (straight-line) distance. The Eucledian distance is found by summing the Eucledian distances of each title from its goal position. 

It is my theory that that since the Manhattan distance gives a more realistic representation of how the titles in the puzzle move it may give a better heuristic, and that is why I have chosen these two functions to see which performs better and which takes less time.

<h4> 1.2.3 Then, you should implement two versions of the A⋆
algorithm in Python to solve the 8-puzzle using the
two heuristic functions you have chosen. </h4>

I have chosen to use the start and end states as that of Figure 1 (given in the exercise)

In [9]:
# Exercise 1.2.3 Solution to solve 8-puzzle using Euclidean and Manhattan functions

# The following lines import the required packages for generating results
import time
# import the numpy package in the np namespace
import numpy as np
# deepcopy constructs a new compound object and then, recursively, 
# inserts copies into it of the objects found in the original
from copy import deepcopy

def getManhattanDistance(pos, goal):
    """
    Function that returns the total Manhattan cost.
    It calculates the distance between where tiles are and where they should be.
    
    Parameters
    ----------
    pos : the current state of the puzzle.
    goal : the final target state of the puzzle.
    """
    
    modulusTotal = abs(pos // 3 - goal // 3)
    remainderTotal = abs(pos % 3 - goal % 3)
    distance = modulusTotal + remainderTotal
    #Return the total Manhattan cost.
    return sum(distance)

def getEuclideanDistance(pos, goal):
    """
    Function that returns the total Euclidean Distance.
    
    Parameters
    ----------
    pos: the current state of the puzzle.
    goal : the final target of the puzzle.
    """
    
    modulusTotal = abs(pos // 3 - goal // 3) ** 2
    remainderTotal = abs(pos % 3 - goal % 3) ** 2
    
    # sum of the squares
    # do squareroot and find Euclidean distance
    distance = np.sqrt(modulusTotal + remainderTotal)
    #Return the total Euclidean cost.
    return sum(distance)

def solve(heurChoice, init_values, goal):

    """
    Method which uses the A* alg to solve for the 8-puzzle problem.

    Keyword arguments:
    heurChoice -- refers to the heurestic chosen (Euclidean or Manhattan) by user via command line input.
    pos -- refers to puzzle's current positions.
    goal -- the goal state of the puzzle.
    """

    # Start the timer
    start = time.time()

    # Make sure to check that inital positions are not equal to goal positions.
    if np.array_equal(init_values, goal):
        return goal, time.time() - start

    # Make the templates for the arrays
    rows_template = np.array([("LEFT", [0, 3, 6], -1),
                      ("RIGHT", [2, 5, 8], 1),
                      ("UP", [0, 1, 2], -3),
                      ("DOWN", [6, 7, 8], 3)],
                     dtype = [("MOVE", str, 1),
                              ("POSITION", list),
                              ("HEAD", int)])
    
    puzzle_template = [("puzzle", list), ("parent", int),
                   ("g_score", int), ("h_score", int)]

    # Use the selected heurestic to solve the puzzle
    
    #If user selects 1 return the Euclidean Distance
    if heurChoice == "1":
        h_score = getEuclideanDistance(init_values, goal)
    #If user selects 2 return the Manhattan Distance
    else:
        h_score = getManhattanDistance(init_values, goal)

    state = np.array([(init_values, -1, 0, h_score)], puzzle_template)

    rank = np.array( [(0, h_score)], [("POSITION", int),("f_score", int)])

    print("Attempting to search for solution...")
    
    solved = False
    
    while not solved:
        # Order the grids based on their f score. 
        rank = np.sort(rank, kind="mergesort",
                           order=["f_score", "POSITION"])
        
        # Choose the grid with the smallest f ranking score.
        position, f_score = rank[0]
        rank = np.delete(rank, 0, 0)
        puzzle, parent, g_score, h_score = state[position]

        # increment the g score if position empty.
        nullPos = int(np.where(puzzle == 0)[0])
        g_score += 1

        # Iterate through each non-null position. 
        for row in rows_template:
            if not nullPos in row["POSITION"]:
                # Copy the current positions of the puzzle.
                availablePos = deepcopy(puzzle)
                
                # Using the 'changeable' variable, swap the positions.
                changeable = availablePos[nullPos + row["HEAD"]]
                availablePos[nullPos + row["HEAD"]] = availablePos[nullPos]
                availablePos[nullPos] = changeable

                # Check to make sure all paths have been traversed.
                if not (np.all(list(state["puzzle"]) == availablePos, 1)).any():
                    
                    # Choose a heuristic.
                    if heurChoice == "1":
                        h_score = euclideanCost(availablePos, goal)
                    else:
                        h_score = getManhattanDistance(availablePos, goal)

                    # Append the remaining values (in the queue) to the numpy array.
                    remaining = np.array([(availablePos, position, g_score, h_score)],
                                     puzzle_template)
                    state = np.append(state, remaining, 0)

                    # Find the final cost (f score) by summing the h score and g score.
                    f_score = h_score + g_score

                    remaining = np.array([(len(state) - 1, f_score)],
                                     [("POSITION", int),("f_score", int)])
                    rank = np.append(rank, remaining, 0)

                    # Check if the puzzle has been solved
                    if np.array_equal(availablePos, goal):
                        solved = True
                        break

    print("Solution has been found. Solved!\n")
    return availablePos, time.time() - start

def showPuzzle(state):
    """Displays the puzzle array in a way that is easier to visualize."""
    print(str(state.reshape(-1, 3, 3))
                    .replace("  [", "")
                    .replace("[", "")
                    .replace("]", "") + "\n")

def main():
    """
    The main function to solve the 8-puzzle problem with a choice of two
    heuristics to be used in an A* algorithm.
    """
    # Use numpy arrays to define the starting values of the puzzle.
    init_values = np.array([7, 2, 4, 5, 0, 6, 8, 3, 1])
    # Use numpy arrays to define the target values of the puzzle.
    goal = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8])

    # Asks the user to pick a heuristic (either Euclidean or Manhattan).
    heurChoice = ""
    while heurChoice != "1" and heurChoice != "2":
        heurChoice = input("Choose a heuristic function:\n1. Euclidean Distance \n2. Manhattan Distance\n")

    # Print the starting values of the puzzle to the screen.
    print("Current Values: \n")
    showPuzzle(init_values)
    #Print the target values of the puzzle to the screen.
    print("Target Values: \n")
    showPuzzle(goal)

    # Solve the puzzle.
    availablePos, time = solve(heurChoice, init_values, goal)

    # Print the final state to the user, and make sure that the values match that of the starting position. 
    # Print the total time taken to solve the puzzle.
    print("Found the Target Values: \n")
    showPuzzle(availablePos)
    print("In: " + str(round(time, 4)) + " seconds")

if __name__ == "__main__":
    main()


Choose a heuristic function:
1. Euclidean Distance 
2. Manhattan Distance
2
Current Values: 

7 2 4
5 0 6
8 3 1

Target Values: 

0 1 2
3 4 5
6 7 8

Attempting to search for solution...
Solution has been found. Solved!

Found the Target Value: 

0 1 2
3 4 5
6 7 8

In: 11.4432 seconds


***
<h4> 1.2.4 Briefly discuss and compare the results given by A⋆ when using the two different heuristic functions in
question 1.2 </h4>

As expected the Manhattan gave the better heuristic in this case as it was able to reach the end state much faster than the Euclidean heuristic. 
The Manhattan took 15.05 seconds, whereas Euclidean required 86.51 seconds. Therefore the Manhattan was around 6 times faster than the Euclidean function, far greater than the time taken for that of the Euclidean. 
In theory it makes sense as the tiles in the 8 puzzle are unable to move diagonally, which would be required for the Euclidean distance, whereas for Manhattan the titles have limited movement so it achieves the shortest distance moved more quickly.
In conclusion, this comparison shows that picking a good heuristic has a significant effect on the time complexity for the A* algorithm.
***

<h4>1.3   Write a general version of the A⋆ algorithm (using either of the two heuristic functions described above) to
solve a generic version of the 8-puzzle where the user can input any start and goal state. (5 marks)

(Hint: can this be done for any generic pair of configurations...?) </h4>

Some of the puzzles are impossible to solve from the initial state. The 8-puzzle has 9! possible configurations, however it has a property wherin only half of the permutations are solveable. Therefore 9!/2 is the total number of solvable configurations.
***

In [13]:
# Exercise 1.2.3 Solution to solve 8-puzzle using Euclidean and Manhattan functions

# The following lines import the required packages for generating results
import time
# import the numpy package in the np namespace
import numpy as np
# deepcopy constructs a new compound object and then, recursively, 
# inserts copies into it of the objects found in the original
from copy import deepcopy

def createThePuzzle(createValues):
    """
    Method which prompts the user to select numbers of their choice for the starting values 
    or target values for the puzzle.
    """

    print("\nChoose the values for the puzzle " + createValues + " inputs, use 0 for a gap .")
    inputs = np.zeros(9, dtype=int)
    #display the values input by the user
    showPuzzle(inputs)
    
    # Traverse through the non-filled arrays.
    for x in np.nditer(inputs, op_flags=['readwrite']):
        validInput = False
        inputNumber = int(input("number: "))
        
        # Check number input by user is valid
        while not validInput:
            if (inputNumber > -1 and inputNumber < 9) and \
            (inputNumber not in inputs or inputNumber == 0):
                validInput = True
            else:
                #Print message to screen.
                print("Only numbers from 0-8 are valid, make sure to not repeat numbers!.")
                showPuzzle(inputs)
                inputNumber = int(input(createValues + " number: "))
        x[...] = inputNumber
        showPuzzle(inputs)
    return inputs


def getManhattanDistance(pos, goal):
    """
    Function that returns the total Manhattan cost.
    It calculates the distance between where tiles are and where they should be.
    
    Parameters
    ----------
    pos : the current state of the puzzle.
    goal : the final target state of the puzzle.
    """
    
    modulusTotal = abs(pos // 3 - goal // 3)
    remainderTotal = abs(pos % 3 - goal % 3)
    distance = modulusTotal + remainderTotal
    #Return the total Manhattan cost.
    return sum(distance)

def getEuclideanDistance(pos, goal):
    """
    Function that returns the total Euclidean Distance.
    
    Parameters
    ----------
    pos: the current state of the puzzle.
    goal : the final target of the puzzle.
    """
    
    modulusTotal = abs(pos // 3 - goal // 3) ** 2
    remainderTotal = abs(pos % 3 - goal % 3) ** 2
    
    # sum of the squares
    # do squareroot and find Euclidean distance
    distance = np.sqrt(modulusTotal + remainderTotal)
    #Return the total Euclidean cost.
    return sum(distance)

def solve(heurChoice, init_values, goal):

    """
    Method which uses the A* alg to solve for the 8-puzzle problem.

    Keyword arguments:
    heurChoice -- refers to the heurestic chosen (Euclidean or Manhattan) by user via command line input.
    pos -- refers to puzzle's current positions.
    goal -- the goal state of the puzzle.
    """

    # Start the timer
    start = time.time()

    # Make sure to check that inital positions are not equal to goal positions.
    if np.array_equal(init_values, goal):
        return goal, time.time() - start

    # Make the templates for the arrays
    rows_template = np.array([("LEFT", [0, 3, 6], -1),
                      ("RIGHT", [2, 5, 8], 1),
                      ("UP", [0, 1, 2], -3),
                      ("DOWN", [6, 7, 8], 3)],
                     dtype = [("MOVE", str, 1),
                              ("POSITION", list),
                              ("HEAD", int)])
    
    puzzle_template = [("puzzle", list), ("parent", int),
                   ("g_score", int), ("h_score", int)]

    # Use the selected heurestic to solve the puzzle
    
    #If user selects 1 return the Euclidean Distance
    if heurChoice == "1":
        h_score = getEuclideanDistance(init_values, goal)
    #If user selects 2 return the Manhattan Distance
    else:
        h_score = getManhattanDistance(init_values, goal)

    state = np.array([(init_values, -1, 0, h_score)], puzzle_template)

    rank = np.array( [(0, h_score)], [("POSITION", int),("f_score", int)])

    print("Attempting to search for solution...")
    
    solved = False
    
    while not solved:
        # Order the grids based on their f score. 
        rank = np.sort(rank, kind="mergesort",
                           order=["f_score", "POSITION"])
        
        # Choose the grid with the smallest f ranking score.
        position, f_score = rank[0]
        rank = np.delete(rank, 0, 0)
        puzzle, parent, g_score, h_score = state[position]

        # increment the g score if position empty.
        nullPos = int(np.where(puzzle == 0)[0])
        g_score += 1

        # Iterate through each non-null position. 
        for row in rows_template:
            if not nullPos in row["POSITION"]:
                # Copy the current positions of the puzzle.
                availablePos = deepcopy(puzzle)
                
                # Using the 'changeable' variable, swap the positions.
                changeable = availablePos[nullPos + row["HEAD"]]
                availablePos[nullPos + row["HEAD"]] = availablePos[nullPos]
                availablePos[nullPos] = changeable

                # Check to make sure all paths have been traversed.
                if not (np.all(list(state["puzzle"]) == availablePos, 1)).any():
                    
                    # Choose a heuristic.
                    if heurChoice == "1":
                        h_score = euclideanCost(availablePos, goal)
                    else:
                        h_score = getManhattanDistance(availablePos, goal)

                    # Append the remaining values (in the queue) to the numpy array.
                    remaining = np.array([(availablePos, position, g_score, h_score)],
                                     puzzle_template)
                    state = np.append(state, remaining, 0)

                    # Find the final cost (f score) by summing the h score and g score.
                    f_score = h_score + g_score

                    remaining = np.array([(len(state) - 1, f_score)],
                                     [("POSITION", int),("f_score", int)])
                    rank = np.append(rank, remaining, 0)

                    # Check if the puzzle has been solved
                    if np.array_equal(availablePos, goal):
                        solved = True
                        break

    print("Solution has been found. Solved!\n")
    return availablePos, time.time() - start

def showPuzzle(state):
    """Displays the puzzle array in a way that is easier to visualize."""
    print(str(state.reshape(-1, 3, 3))
                    .replace("  [", "")
                    .replace("[", "")
                    .replace("]", "") + "\n")

def main():
    """
    The main function to solve the 8-puzzle problem with a choice of two
    heuristics to be used in an A* algorithm.
    """
    # User defined initial and goal states of the puzzle as numpy arrays
    init_values = createThePuzzle("Start")
    goal = createThePuzzle("Goal")

    # Asks the user to pick a heuristic (either Euclidean or Manhattan).
    heurChoice = ""
    while heurChoice != "1" and heurChoice != "2":
        heurChoice = input("Choose a heuristic function:\n1. Euclidean Distance \n2. Manhattan Distance\n")

    # Print the starting values of the puzzle to the screen.
    print("Current Values: \n")
    showPuzzle(init_values)
    #Print the target values of the puzzle to the screen.
    print("Target Values: \n")
    showPuzzle(goal)

    # Solve the puzzle.
    availablePos, time = solve(heurChoice, init_values, goal)

    # Print the final state to the user, and make sure that the values match that of the starting position. 
    # Print the total time taken to solve the puzzle.
    print("Found the Target Values: \n")
    showPuzzle(availablePos)
    print("In: " + str(round(time, 4)) + " seconds")

if __name__ == "__main__":
    main()



Choose the values for the puzzle Start inputs, use 0 for a gap .
0 0 0
0 0 0
0 0 0

number: 1
1 0 0
0 0 0
0 0 0

number: 2
1 2 0
0 0 0
0 0 0

number: 3
1 2 3
0 0 0
0 0 0

number: 4
1 2 3
4 0 0
0 0 0

number: 5
1 2 3
4 5 0
0 0 0

number: 6
1 2 3
4 5 6
0 0 0

number: 7
1 2 3
4 5 6
7 0 0

number: 8
1 2 3
4 5 6
7 8 0

number: 0
1 2 3
4 5 6
7 8 0


Choose the values for the puzzle Goal inputs, use 0 for a gap .
0 0 0
0 0 0
0 0 0

number: 0
0 0 0
0 0 0
0 0 0

number: 1
0 1 0
0 0 0
0 0 0

number: 2
0 1 2
0 0 0
0 0 0

number: 3
0 1 2
3 0 0
0 0 0

number: 4
0 1 2
3 4 0
0 0 0

number: 5
0 1 2
3 4 5
0 0 0

number: 6
0 1 2
3 4 5
6 0 0

number: 7
0 1 2
3 4 5
6 7 0

number: 9
Only numbers from 0-8 are valid, make sure to not repeat numbers!.
0 1 2
3 4 5
6 7 0

Goal number: 9
Only numbers from 0-8 are valid, make sure to not repeat numbers!.
0 1 2
3 4 5
6 7 0

Goal number: 8
0 1 2
3 4 5
6 7 8

Choose a heuristic function:
1. Euclidean Distance 
2. Manhattan Distance
2
Current Values: 

1 2 3
4 5 6
7